In [1]:
# goals:
# run VQE on all backends
# run Quantum Volume simulator on all backends
# figure out entanglement scheme on all backends

# so how to transpile VQE scheme for backends?

- generate test circuit (1 cell)

In [2]:
import numpy as np
from qiskit import QuantumCircuit

def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit
circuit = test_circuit()

# IBM Devices
- import qiskit modules
- transpile circuit

In [3]:
# ibm backends:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeBrisbane, FakeOsaka, FakeKyoto, FakeSherbrooke

backend = AerSimulator.from_backend(FakeBrisbane())
# all these backends have 127 qubits.

In [4]:
# ibm transpiling code: (from VQE qiskit docs)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
ibm_circuit_isa = pm.run(circuit)

# from some https://github.com/Qiskit/qiskit/issues/9717
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_ibm_runtime import QiskitRuntimeService

# service = QiskitRuntimeService()
backend = FakeBrisbane() # or service.get_backend("ibm_lagos")
ibm_circuit = transpile(circuit, backend=backend)  # throws error

# ibm_circuit_isa.draw(output='mpl')
# ibm_circuit.draw(output='mpl')
":trol:"

':trol:'

# Braket Devices
- Transpiling for braket (2 cells)
- Transpiling for ionq harmony (2 cells)

In [5]:
import qbraid.runtime
from qbraid.programs import QPROGRAM_REGISTRY
from qbraid.interface import (
    circuits_allclose,
    assert_allclose_up_to_global_phase,
    random_circuit,
)
from qbraid.transpiler import transpile


QPROGRAM_REGISTRY

{'cirq': cirq.circuits.circuit.Circuit,
 'qiskit': qiskit.circuit.quantumcircuit.QuantumCircuit,
 'braket': braket.circuits.circuit.Circuit,
 'braket_ahs': braket.ahs.analog_hamiltonian_simulation.AnalogHamiltonianSimulation,
 'openqasm3': openqasm3.ast.Program,
 'qasm2': str,
 'qasm3': str,
 'ionq': ~IonQDict}

In [6]:
braket_circuit = transpile(circuit, "braket")
print(braket_circuit)

"""
You may need to `pip install qiskit-braket-provider`. However, from the docs:

While the qiskit-braket-provider package offers direct conversions, this comes with a trade-off.
It eliminates intermediate steps but tends to be more prone to errors, particularly with unsupported
gates, compared to the qBraid transpiler. To leverage the best of both, you can integrate the
qiskit-braket-provider's direct conversion into the qBraid transpiler's conversion graph.
Consequently, when converting from Braket to Qiskit, the transpiler will first try the direct
conversion through the qiskit-braket-provider. If this conversion fails, it will automatically
revert to one of the other native qBraid conversion paths
"""

T  : │  0  │  1  │  2   │       3       │    4    │        5        │       6       │
      ┌───┐ ┌───┐ ┌───┐   ┌──────────┐     ┌───┐                                     
q0 : ─┤ H ├─┤ X ├─┤ S ├───┤ Rx(0.79) ├─────┤ V ├──────x─────────────────────●────────
      └───┘ └───┘ └───┘   └──────────┘     └───┘      │                     │        
      ┌───┐ ┌───┐ ┌────┐  ┌──────────┐    ┌────┐      │                   ┌─┴─┐      
q1 : ─┤ H ├─┤ X ├─┤ Si ├──┤ Ry(1.57) ├────┤ Vi ├──────┼────────x──────────┤ X ├──────
      └───┘ └───┘ └────┘  └──────────┘    └────┘      │        │          └───┘      
      ┌───┐ ┌───┐ ┌───┐   ┌──────────┐   ┌───────┐    │        │                     
q2 : ─┤ H ├─┤ Y ├─┤ T ├───┤ Rz(2.36) ├───┤ ISWAP ├────x────────┼────────────●────────
      └───┘ └───┘ └───┘   └──────────┘   └───┬───┘             │            │        
      ┌───┐ ┌───┐ ┌────┐ ┌─────────────┐ ┌───┴───┐             │     ┌──────┴──────┐ 
q3 : ─┤ H ├─┤ Z ├─┤ Ti ├─┤ PHASE(0.39) ├─┤ ISWAP ├────

"\nYou may need to `pip install qiskit-braket-provider`. However, from the docs:\n\nWhile the qiskit-braket-provider package offers direct conversions, this comes with a trade-off.\nIt eliminates intermediate steps but tends to be more prone to errors, particularly with unsupported\ngates, compared to the qBraid transpiler. To leverage the best of both, you can integrate the\nqiskit-braket-provider's direct conversion into the qBraid transpiler's conversion graph.\nConsequently, when converting from Braket to Qiskit, the transpiler will first try the direct\nconversion through the qiskit-braket-provider. If this conversion fails, it will automatically\nrevert to one of the other native qBraid conversion paths\n"

In [7]:
from qbraid.transpiler import transpile

qasm3_program = transpile(circuit, 'cirq')

from qbraid.programs import load_program
qprogram = load_program(qasm3_program)
old_program = qprogram.program

from qbraid.runtime.ionq import IonQProvider
provider = IonQProvider('')
device = provider.get_device("qpu.harmony")

try:
    print(qprogram.transform(device))
except Exception as e:
    print(e)
print(old_program == qprogram.program)
qprogram.program

None
True


q_0: ───H───X───S──────Rx(0.25π)───X^0.5────────────────×───────@────────
                                                        │       │
q_1: ───H───X───S^-1───Ry(0.5π)────X^-0.5───────────────┼───×───X────────
                                                        │   │
q_2: ───H───Y───T──────Rz(0.75π)───S────────H───@───X───×───┼───@────────
                                                │   │       │   │
q_3: ───H───Z───T^-1───Z^(1/8)─────S────────────X───@───H───×───@^0.25───

In [8]:
import qiskit.qasm3
circuit = qiskit.qasm3.loads(qprogram.program)
circuit.draw("mpl")

TypeError: ord() expected string of length 1, but Moment found

# Generic Backend List

In [22]:
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeBrisbane, FakeOsaka, FakeKyoto, FakeSherbrooke
from qbraid.transpiler import transpile
from qbraid.programs import load_program


# brisbane, osaka are the same
# kyoto and sherbrooke are each different?
IBM_BACKENDS = {
    "brisbane": FakeBrisbane(),
    "osaka": FakeOsaka(),
    "kyoto": FakeKyoto(),
    "sherbrooke": FakeSherbrooke()
}

AWS_BACKENDS = {
    "aspen": {
        'aws_id': "arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3",
    },
    "harmony": {},
    "aria1": {},
    "aria2": {},
    "forte1": {},
    "garnet": {
        'aws_id': "arn:aws:braket:eu-north-1::device/qpu/iqm/Garnet",
    },
    "ankaa": {
        'aws_id': "arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-2",
    }
}


def get_simulator(backend):
    backend = backend.lower()
    if backend in IBM_BACKENDS:
        return AerSimulator.from_backend(IBM_BACKENDS[backend])

    if backend == "aquila":
        raise("AUAHSDFHADSHFA do VQE first")
    
    if backend in AWS_BACKENDS:
        # return a base AerSimulator with noise models
        return None

def prepare_circuit(circuit, backend):
    backend = backend.lower()
    if backend in IBM_BACKENDS:
        from qiskit import transpile
        # non-optimized transpilation:
        return transpile(circuit, backend=get_simulator(backend))

    if backend == "aquila":
        raise("Sorry, I have no idea how to help you there.")

    if backend in AWS_BACKENDS:
        from qbraid.transpiler import transpile
        braket_program = transpile(circuit, 'braket')
        qprogram = load_program(braket_program)
        if backend in "harmonyaria1aria2forte1":
            from qbraid.runtime.ionq import IonQProvider
            provider = IonQProvider('')
            device = provider.get_device("qpu." + backend)
        else:
            from qbraid.runtime.braket import BraketProvider
            provider = BraketProvider('')
            device = provider.get_device(AWS_BACKENDS[backend]["aws_id"])

        qprogram.transform(device)
        return transpile(qprogram, "qiskit")
        
    raise("Please check backend named", backend)


circuit = test_circuit()

In [23]:
for backend in AWS_BACKENDS:
    display(prepare_circuit(circuit, backend).draw(output='mpl'))

NoCredentialsError: Unable to locate credentials

# VQE on all backends